In [1]:
from keras.datasets import imdb

Using TensorFlow backend.


In [27]:
imdb.load_data?

Signature:
imdb.load_data(
    ["path='imdb.npz'", 'num_words=None', 'skip_top=0', 'maxlen=None', 'seed=113', 'start_char=1', 'oov_char=2', 'index_from=3', '**kwargs'],
)
Docstring:
Loads the IMDB dataset.

# Arguments
    path: where to cache the data (relative to `~/.keras/dataset`).
    num_words: max number of words to include. Words are ranked
        by how often they occur (in the training set) and only
        the most frequent words are kept
    skip_top: skip the top N most frequently occurring words
        (which may not be informative).
    maxlen: sequences longer than this will be filtered out.
    seed: random seed for sample shuffling.
    start_char: The start of a sequence will be marked with this character.
        Set to 1 because 0 is usually the padding character.
    oov_char: words that were cut out because of the `num_words`
        or `skip_top` limit will be replaced with this character.
    index_from: index actual words with this index and higher.

# Retur

In [2]:
! wget https://s3.amazonaws.com/text-datasets/imdb.npz

--2019-11-03 15:26:38--  https://s3.amazonaws.com/text-datasets/imdb.npz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.170.133
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.170.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17464789 (17M) [application/octet-stream]
Saving to: ‘imdb.npz’

imdb.npz            100%[===================>]  16.66M  2.70MB/s    in 22s     

2019-11-03 15:27:02 (774 KB/s) - ‘imdb.npz’ saved [17464789/17464789]



In [3]:
import numpy as np

In [4]:
imdb_files = np.load("imdb.npz", allow_pickle=True)

In [5]:
imdb_files.files

['x_test', 'x_train', 'y_train', 'y_test']

In [6]:
x_train = imdb_files['x_train']
x_test = imdb_files['x_test']
y_train = imdb_files['y_train']
y_test = imdb_files['y_test']

In [7]:
vocab = imdb.get_word_index()

In [8]:
rev_vocab = {}

for key, val in vocab.items():
    rev_vocab[val] = key

In [10]:
map(lambda idx : rev_vocab[idx], x_train[0])

In [11]:
line = " ".join(map(lambda idx : rev_vocab[idx], x_train[0]))

In [12]:
line

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [14]:
size = 10000

In [15]:
X_train_fil = []
for review in x_train:
    X_train_fil.append(list(filter(lambda word : word < size, review)))
  
X_test_fil = []
for review in x_test:
    X_test_fil.append(list(filter(lambda word : word < size, review)))

In [16]:
len(X_train_fil[0])

127

In [17]:
from keras.preprocessing import sequence

In [18]:
X_train_gen = sequence.pad_sequences(X_train_fil, maxlen=300)
X_test_gen = sequence.pad_sequences(X_test_fil, maxlen=300)

In [19]:
type(X_train_gen)

numpy.ndarray

In [20]:
X_test_gen.shape

(25000, 300)

In [21]:
from keras.layers import Embedding, SimpleRNN, Dense
from keras.models import Sequential

In [22]:
model = Sequential()
model.add(Embedding(size, 200))
model.add(SimpleRNN(32, dropout=.2, recurrent_dropout=.2))
model.add(Dense(1, activation="sigmoid"))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         2000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                7456      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,007,489
Trainable params: 2,007,489
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])

In [25]:
model.fit(X_train_gen, y_train, epochs=6)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/6
18848/25000 [=====================>........] - ETA: 20s - loss: 0.6882 - accuracy: 0.5420

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test_gen, y_test)